In [1]:
import pandas as pd
import os

In [2]:
SCRIPT_DIR_PATH = os.getcwd()
DATA_DIR_PATH = os.path.join(SCRIPT_DIR_PATH, "data")
OUTPUT_DIR_PATH = os.path.join(SCRIPT_DIR_PATH, "output")

## Load Inventory and Crosswalk Files

In [3]:
# Set up file names
INVENTORY_FILE = os.path.join(DATA_DIR_PATH, "CSC-GHG_emissions-April2024_to_calibrate.csv")
CROSSWALK_FILE = os.path.join(DATA_DIR_PATH, "sisepuede_edgar_active_crosswalk.csv")

# Set up region name, year, and iso3
REGION = "morocco"
YEAR = 2022
ISO3 = "MAR"

In [4]:
inventory_df = pd.read_csv(INVENTORY_FILE, encoding='latin1')
crosswalk_df = pd.read_csv(CROSSWALK_FILE)

In [5]:
inventory_df.head()

,Code,Country,EDGAR Country Code,Income group,Lending category,Region,CSC Sector,CSC Subsector,Gas,Units,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,Aruba,ABW,High income,NaN,Latin America & Caribbean,Energy,EN - Building,CH4,MtCO2e,...,0.000554,0.000538,0.000500,0.000570,0.000542,0.000574,0.000471,0.000616,0.000617,0.000622
1,ABW,Aruba,ABW,High income,NaN,Latin America & Caribbean,Energy,EN - Building,CO2,MtCO2e,...,0.047090,0.037332,0.035079,0.039403,0.035488,0.044616,0.050051,0.044068,0.042466,0.044068
2,ABW,Aruba,ABW,High income,NaN,Latin America & Caribbean,Energy,EN - Building,N2O,MtCO2e,...,0.000160,0.000138,0.000130,0.000137,0.000135,0.000235,0.000264,0.000240,0.000234,0.000241
3,ABW,Aruba,ABW,High income,NaN,Latin America & Caribbean,Energy,EN - Electricity/Heat,CH4,MtCO2e,...,0.000114,0.000120,0.000126,0.000127,0.000122,0.000132,0.000156,0.000135,0.000132,0.000136
4,ABW,Aruba,ABW,High income,NaN,Latin America & Caribbean,Energy,EN - Electricity/Heat,CO2,MtCO2e,...,0.213816,0.222204,0.229352,0.239740,0.223555,0.232325,0.274290,0.232537,0.224081,0.232537


In [6]:
crosswalk_df.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,ignore,note,need_better_information_on_what_is_contained
0,agrc,ch4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,NaN,may include fertilizer application,1.0
1,agrc,co2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,NaN,may include liming and urea,1.0
2,agrc,n2o,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,NaN,may include fertilizer application,1.0
3,lvst,ch4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,NaN,NaN,NaN
4,lsmm,ch4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,NaN,NaN,NaN


## Cleaning dataframes before merge

In [7]:
# Filter inventory for the specified region and year
inventory_region_year_df = inventory_df[(inventory_df['Code'] == ISO3)][["Code", "CSC Subsector", "Gas", str(YEAR)]]
inventory_region_year_df = inventory_region_year_df.rename(columns={str(YEAR): ISO3})
inventory_region_year_df.head()

,Code,CSC Subsector,Gas,MAR
3279,MAR,AG - Livestock,CH4,9.155089
3280,MAR,AG - Livestock,N2O,0.061910
3281,MAR,AG - Crops,CH4,0.026607
3282,MAR,AG - Crops,CO2,0.611888
3283,MAR,AG - Crops,N2O,4.621083


In [8]:
# Make Gas column Uppercase for merging
crosswalk_df['Gas'] = crosswalk_df['Gas'].str.upper()
inventory_region_year_df['Gas'] = inventory_region_year_df['Gas'].str.upper()

In [9]:
crosswalk_df = crosswalk_df.drop(columns=["ignore", "note", "need_better_information_on_what_is_contained"])
crosswalk_df.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars
0,agrc,CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...
1,agrc,CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...
2,agrc,N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...


In [10]:
# Let's create a id column to merge on
inventory_region_year_df["id"] = inventory_region_year_df["CSC Subsector"] + " - "+ inventory_region_year_df["Gas"]
crosswalk_df["id"] = crosswalk_df["Edgar_Subsector"] + " - " + crosswalk_df["Gas"]

In [11]:
inventory_region_year_df.head()

,Code,CSC Subsector,Gas,MAR,id
3279,MAR,AG - Livestock,CH4,9.155089,AG - Livestock - CH4
3280,MAR,AG - Livestock,N2O,0.061910,AG - Livestock - N2O
3281,MAR,AG - Crops,CH4,0.026607,AG - Crops - CH4
3282,MAR,AG - Crops,CO2,0.611888,AG - Crops - CO2
3283,MAR,AG - Crops,N2O,4.621083,AG - Crops - N2O


In [12]:
crosswalk_df.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id
0,agrc,CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,AG - Crops - CH4
1,agrc,CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,AG - Crops - CO2
2,agrc,N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,AG - Crops - N2O
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4


In [13]:
# Check for duplicated ids in both dataframes
print("Duplicated IDs in inventory_region_year_df:", inventory_region_year_df['id'].duplicated().sum())
print("Duplicated IDs in crosswalk_df:", crosswalk_df['id'].duplicated().sum())

Duplicated IDs in inventory_region_year_df: 0
Duplicated IDs in crosswalk_df: 2


## Merge Data

In [14]:
inventory_region_year_df.columns

Index(['Code', 'CSC Subsector', 'Gas', 'MAR', 'id'], dtype='object')

In [15]:
merged_df = pd.merge(crosswalk_df, inventory_region_year_df[["id", ISO3]], how='left', on='id')
merged_df.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR
0,agrc,CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,AG - Crops - CH4,0.026607
1,agrc,CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,AG - Crops - CO2,0.611888
2,agrc,N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,AG - Crops - N2O,4.621083
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,9.155089
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,9.155089


In [16]:
# Check duplicated ids
duplicated_ids = merged_df[merged_df['id'].duplicated(keep=False)]
duplicated_ids

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,9.155089
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,9.155089
62,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Removals,emission_co2e_co2_entc_bmass_processing_and_re...,LULUCF - Forest Land - CO2,-1.750000
63,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Sequestration,emission_co2e_co2_frst_sequestration_mangroves...,LULUCF - Forest Land - CO2,-1.750000


## Deal with NaNs

In [17]:
# merged_df[merged_df[ISO3].isna()]

### Check IPPU

In [18]:
inventory_region_year_df[inventory_region_year_df["CSC Subsector"] == "IN - Industrial Processes"]

,Code,CSC Subsector,Gas,MAR,id
3299,MAR,IN - Industrial Processes,CH4,0.000000,IN - Industrial Processes - CH4
3300,MAR,IN - Industrial Processes,CO2,5.353567,IN - Industrial Processes - CO2
3301,MAR,IN - Industrial Processes,HFC,1.817981,IN - Industrial Processes - HFC
3302,MAR,IN - Industrial Processes,N2O,0.462577,IN - Industrial Processes - N2O


In [19]:
pd.merge(crosswalk_df[crosswalk_df["Edgar_Subsector"] == "IN - Industrial Processes"], 
        inventory_region_year_df[inventory_region_year_df["CSC Subsector"] == "IN - Industrial Processes"],
        how="inner",
        on="id"
        )

,Subsector,Gas_x,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,Code,CSC Subsector,Gas_y,MAR
0,ippu,CH4,Industrial Processes,IN - Industrial Processes,IN - Industrial Processes,emission_co2e_ch4_ippu_production_chemicals:em...,IN - Industrial Processes - CH4,MAR,IN - Industrial Processes,CH4,0.000000
1,ippu,CO2,Industrial Processes,IN - Industrial Processes,IN - Industrial Processes,emission_co2e_co2_ippu_product_use_product_use...,IN - Industrial Processes - CO2,MAR,IN - Industrial Processes,CO2,5.353567
2,ippu,N2O,Industrial Processes,IN - Industrial Processes,IN - Industrial Processes,emission_co2e_n2o_ippu_production_chemicals:em...,IN - Industrial Processes - N2O,MAR,IN - Industrial Processes,N2O,0.462577


In [20]:
# Get the emission value for HFC gases
hfc_gas_total = inventory_region_year_df[(inventory_region_year_df["CSC Subsector"] == "IN - Industrial Processes") & (inventory_region_year_df["Gas"] == "HFC")][ISO3].values[0]
hfc_gas_total

np.float64(1.817981221)

In [21]:
merged_df.loc[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"] == "HFCS"), ISO3] = hfc_gas_total
merged_df[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"] == "HFCS")]

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR
54,ippu,HFCS,Industrial Processes,IN - Industrial Processes,IN - Industrial Processes,emission_co2e_hfcs_ippu_product_use_product_us...,IN - Industrial Processes - HFCS,1.817981


In [22]:
# # Get the emission value for PFC gases
# pfc_gas_total = inventory_region_year_df[(inventory_region_year_df["CSC Subsector"] == "IN - Industrial Processes") & (inventory_region_year_df["Gas"] == "PFC")][ISO3].values[0]
# pfc_gas_total

In [23]:
# merged_df.loc[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"] == "PFCS"), ISO3] = pfc_gas_total
# merged_df[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"] == "PFCS")]

In [24]:
# hcfc_gas_total = inventory_region_year_df[(inventory_region_year_df["CSC Subsector"] == "IN - Industrial Processes") & (inventory_region_year_df["Gas"] == "HCFC")][ISO3].values[0]
# hcfc_gas_total

In [25]:
# hcfc_gases = [
#     "CH3CCI2F",  # HCFC-21
#     "CH3CCIF2"   # HCFC-22
# ]

# for gas in hcfc_gases:
#     merged_df.loc[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"] == gas), ISO3] = hcfc_gas_total / len(hcfc_gases)

# merged_df[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"].isin(hcfc_gases))]


In [26]:
merged_df[ISO3] = merged_df[ISO3].fillna(0)
merged_df.isna().sum()

Subsector                    0
Gas                          0
Edgar_Sector                 0
Edgar_Subsector              0
Edgar_Subsector_Synthetic    0
Vars                         0
id                           0
MAR                          0
dtype: int64

In [27]:
print("Original inventory total:", inventory_region_year_df[ISO3].sum())
print("Merged dataframe total:", merged_df[ISO3].sum())

Original inventory total: 113.020703261
Merged dataframe total: 120.42579255200002


## Fix discrepancies between totals in original inventory and merged inventory

In [28]:
duplicated_ids.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,9.155089
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,9.155089
62,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Removals,emission_co2e_co2_entc_bmass_processing_and_re...,LULUCF - Forest Land - CO2,-1.750000
63,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Sequestration,emission_co2e_co2_frst_sequestration_mangroves...,LULUCF - Forest Land - CO2,-1.750000


In [29]:
duplicated_ids.id.unique()

array(['AG - Livestock - CH4', 'LULUCF - Forest Land - CO2'], dtype=object)

In [30]:
merged_df[merged_df.id.isin(duplicated_ids.id.unique())]

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,9.155089
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,9.155089
62,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Removals,emission_co2e_co2_entc_bmass_processing_and_re...,LULUCF - Forest Land - CO2,-1.750000
63,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Sequestration,emission_co2e_co2_frst_sequestration_mangroves...,LULUCF - Forest Land - CO2,-1.750000


In [31]:
# Since we had duplicated ids in the crosswalk, we need to distribute the emissions values equally among the duplicated rows
lvst_ch4_value = merged_df[(merged_df["Subsector"] == "lvst") & (merged_df["Gas"] == "CH4")][ISO3].values[0]
lvst_ch4_value

np.float64(9.155089291)

In [32]:
value_to_distribute = lvst_ch4_value / 2
value_to_distribute

np.float64(4.5775446455)

In [33]:
# Update values
merged_df.loc[merged_df.id == "AG - Livestock - CH4", ISO3] = value_to_distribute

In [34]:
frst_co2_value = merged_df[(merged_df["Subsector"] == "frst") & (merged_df["Gas"] == "CO2")][ISO3].values[0]
frst_co2_value

np.float64(-1.75)

In [35]:
frst_co2_value_to_distribute = frst_co2_value / 2
frst_co2_value_to_distribute

np.float64(-0.875)

In [36]:
# Update values
merged_df.loc[merged_df.id == "LULUCF - Forest Land - CO2", ISO3] = frst_co2_value_to_distribute

In [37]:
merged_df[merged_df.id.isin(duplicated_ids.id.unique())]

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,4.577545
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,4.577545
62,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Removals,emission_co2e_co2_entc_bmass_processing_and_re...,LULUCF - Forest Land - CO2,-0.875000
63,frst,CO2,"Land Use, Land Use Change, and Forestry",LULUCF - Forest Land,LULUCF - Forest Land Sequestration,emission_co2e_co2_frst_sequestration_mangroves...,LULUCF - Forest Land - CO2,-0.875000


In [38]:
print("Original inventory total:", inventory_region_year_df[ISO3].sum())
print("Merged dataframe total:", merged_df[ISO3].sum())

Original inventory total: 113.020703261
Merged dataframe total: 113.020703261


## Double check IPPU

In [39]:
inventory_region_year_df[inventory_region_year_df["CSC Subsector"] == "IN - Industrial Processes"]

,Code,CSC Subsector,Gas,MAR,id
3299,MAR,IN - Industrial Processes,CH4,0.000000,IN - Industrial Processes - CH4
3300,MAR,IN - Industrial Processes,CO2,5.353567,IN - Industrial Processes - CO2
3301,MAR,IN - Industrial Processes,HFC,1.817981,IN - Industrial Processes - HFC
3302,MAR,IN - Industrial Processes,N2O,0.462577,IN - Industrial Processes - N2O


In [40]:
merged_df[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (~merged_df["Gas"].isin(["CH4", "CO2", "N2O", "SF6"]))][ISO3].sum()

np.float64(1.817981221)

In [41]:
merged_df[(merged_df["Edgar_Subsector"] == "IN - Industrial Processes") & (merged_df["Gas"].isin(["CH4", "CO2", "N2O", "SF6"]))][ISO3]

49    0.000000
53    5.353567
55    0.462577
58    0.000000
Name: MAR, dtype: float64

## Save

In [42]:
merged_df["Edgar_Class"] = merged_df["Edgar_Subsector"] + ":" +merged_df["Gas"] 
merged_df.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR,Edgar_Class
0,agrc,CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,AG - Crops - CH4,0.026607,AG - Crops:CH4
1,agrc,CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,AG - Crops - CO2,0.611888,AG - Crops:CO2
2,agrc,N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,AG - Crops - N2O,4.621083,AG - Crops:N2O
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,4.577545,AG - Livestock:CH4
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,4.577545,AG - Livestock:CH4


In [43]:
merged_df.to_csv(os.path.join(OUTPUT_DIR_PATH, f"emission_targets_{REGION}_{YEAR}.csv"), index=False)